# Julia 機器學習：GLM 線性迴歸

## 作業 027：波士頓房價預測資料集

請使用 GLM 中的模型，建立一個預測模型來預測波士頓的房價。

In [1]:
using GLM, RDatasets, MLDataUtils

## 讀取資料

In [2]:
boston = dataset("MASS", "Boston")
first(boston, 10)

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311


In [3]:
indecies = MLDataUtils.shuffleobs(collect(1:nrow(boston)))
train_ind, test_ind = MLDataUtils.splitobs(indecies, at = 0.8);

In [5]:
train = boston[train_ind, :]
test = boston[test_ind, :]

,Crim,Zn,Indus,Chas,NOx,Rm,Age,Dis,Rad,Tax
,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64
1,7.52601,0.0,18.1,0,0.713,6.417,98.3,2.185,24,666
2,0.1146,20.0,6.96,0,0.464,6.538,58.7,3.9175,3,223
3,7.99248,0.0,18.1,0,0.7,5.52,100.0,1.5331,24,666
4,0.26938,0.0,9.9,0,0.544,6.266,82.8,3.2628,4,304
5,6.39312,0.0,18.1,0,0.584,6.162,97.4,2.206,24,666
6,0.25387,0.0,6.91,0,0.448,5.399,95.3,5.87,3,233
7,0.10612,30.0,4.93,0,0.428,6.095,65.1,6.3361,6,300
8,0.03738,0.0,5.19,0,0.515,6.31,38.5,6.4584,5,224
9,4.75237,0.0,18.1,0,0.713,6.525,86.5,2.4358,24,666


In [6]:
# 多元線性回歸

ols = GLM.lm(@formula(Rm ~ Crim + Zn + Indus + Chas + NOx + Age + Dis + Rad + Tax), train)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Rm ~ 1 + Crim + Zn + Indus + Chas + NOx + Age + Dis + Rad + Tax

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────
                 Estimate   Std. Error    t value  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────────
(Intercept)   7.61655      0.332165     22.93        <1e-73   6.96352      8.26958   
Crim         -0.0135802    0.00471113   -2.88258     0.0042  -0.0228422   -0.00431818
Zn            0.00838921   0.00188489    4.45077     <1e-4    0.00468354   0.0120949 
Indus        -0.0386986    0.00905374   -4.27433     <1e-4   -0.0564982   -0.0208991 
Chas          0.228968     0.129686      1.76555     0.0782  -0.0259934    0.483929  
NOx          -0.740763     0.528837     -1.40074     0.1621  -1

In [7]:
# 預測

predict(ols, test)

101-element Array{Union{Missing, Float64},1}:
 6.011130908941406 
 6.622871704506615 
 6.086456383391529 
 6.307494810878582 
 6.1193485831659915
 6.2614560575189415
 6.546690378385247 
 6.245968322476925 
 6.01574204179921  
 6.282263833386057 
 6.245593892428455 
 5.889248193906975 
 6.824487042477704 
 ⋮                 
 6.627257268488772 
 5.856359516565888 
 6.102172819869478 
 6.597007945237302 
 6.268751520004027 
 5.932011821457046 
 6.848263750101788 
 6.781275408467307 
 6.579532226210794 
 6.152145592937555 
 6.818657540439462 
 5.943323898352177 